In [96]:
'''
Example: Supervised learning with the Naive Bayes algorithm.
'''

'\nExample: Supervised learning with the Naive Bayes algorithm.\n'

In [97]:
from __future__ import print_function

import numpy as np

import recordlinkage as rl
from recordlinkage.index import Block
from recordlinkage.compare import Exact, String

In [98]:
from recordlinkage.datasets import load_febrl3

'''
This data set contains 5000 records (2000 originals and 3000 duplicates), 
with a maximum of 5 duplicates based on one original record (and a Zipf distribution of duplicate records). 
Distribution of duplicates: 
    168 originals records have 5 duplicate records 
    161 originals records have 4 duplicate records 
    212 originals records have 3 duplicate records 
    256 originals records have 2 duplicate records 
    368 originals records have 1 duplicate record 1
    835 originals records have no duplicate record   
'''

'\nThis data set contains 5000 records (2000 originals and 3000 duplicates), \nwith a maximum of 5 duplicates based on one original record (and a Zipf distribution of duplicate records). \nDistribution of duplicates: \n    168 originals records have 5 duplicate records \n    161 originals records have 4 duplicate records \n    212 originals records have 3 duplicate records \n    256 originals records have 2 duplicate records \n    368 originals records have 1 duplicate record 1\n    835 originals records have no duplicate record   \n'

In [99]:
# set logging
rl.logging.set_verbosity(rl.logging.INFO)

In [100]:
# load dataset
print('Loading data...')
dfA, true_links = load_febrl3(return_links=True)
print(len(dfA), 'records in dataset A')
print(len(true_links), 'links in dataset A')

Loading data...
5000 records in dataset A
6538 links in dataset A


In [101]:
# start indexing
print('Build index...')
indexer = rl.Index()
indexer.add(Block('given_name'))
indexer.add(Block('surname'))
indexer.add(Block('soc_sec_id'))
candidate_links = indexer.index(dfA)
print(len(candidate_links), 'candidate links in dataset A')

Build index...
INFO:recordlinkage:indexing - initialize Index class
INFO:recordlinkage:indexing [1/?] - time: 0.79s - pairs: 75984/12497500 - rr: 0.99392
INFO:recordlinkage:indexing [1/?] - time: 0.79s - pairs_total: 75984/12497500 - rr_total: 0.99392
75984 candidate links in dataset A


In [102]:
# start comparing
print('Start comparing...')
comparer = rl.Compare()
comparer.add(Exact('given_name', 'given_name', label='given_name'))
comparer.add(String('surname', 'surname', method='jarowinkler',
                    threshold=0.85, label='surname'))
comparer.add(Exact('date_of_birth', 'date_of_birth', label='date_of_birth'))
comparer.add(Exact('suburb', 'suburb', label='suburb'))
comparer.add(Exact('state', 'state', label='state'))
comparer.add(String('address_1', 'address_1', threshold=0.85,
                    label='address_1'))
comparer.add(String('address_2', 'address_2', threshold=0.85,
                    label='address_2'))
features = comparer.compute(candidate_links, dfA)

print('feature shape', features.shape)

Start comparing...
INFO:recordlinkage:comparing - initialize Compare class
INFO:recordlinkage:comparing [1/?] - time: 1.45s - pairs: 75984
INFO:recordlinkage:comparing [1/?] - time: 1.45s - pairs_total: 75984
feature shape (75984, 7)


In [103]:
# Initialise the NaiveBayesClassifier.
cl = rl.NaiveBayesClassifier()
cl.fit(features, true_links)

INFO:recordlinkage:Classification - start training NaiveBayesClassifier
INFO:recordlinkage:Classification - training computation time: ~0.33s


In [104]:
# Print the parameters that are trained (m, u and p). Note that the estimates
# are very good.
print("p probability P(Match):", cl.p)
print("m probabilities P(x_i=1|Match):", cl.m_probs)
print("u probabilities P(x_i=1|Non-Match):", cl.u_probs)
print("log m probabilities P(x_i=1|Match):", cl.log_m_probs)
print("log u probabilities P(x_i=1|Non-Match):", cl.log_u_probs)
print("log weights of features:", cl.log_weights)
print("weights of features:", cl.weights)

p probability P(Match): 0.08287271004421981
m probabilities P(x_i=1|Match): {'given_name': {0.0: 0.42512307685808887, 1.0: 0.5748769231419115}, 'surname': {0.0: 0.2002540987691251, 1.0: 0.7997459012308754}, 'date_of_birth': {0.0: 0.13593775969706975, 1.0: 0.8640622403029314}, 'suburb': {0.0: 0.37621089800822977, 1.0: 0.6237891019917708}, 'state': {0.0: 0.0852787173168583, 1.0: 0.9147212826831417}, 'address_1': {0.0: 0.19088455801541823, 1.0: 0.809115441984583}, 'address_2': {0.0: 0.416547564981815, 1.0: 0.5834524350181852}}
u probabilities P(x_i=1|Non-Match): {'given_name': {0.0: 0.48118013408188043, 1.0: 0.5188198659181207}, 'surname': {0.0: 0.516394736417425, 1.0: 0.48360526358257583}, 'date_of_birth': {0.0: 0.9999999985650128, 1.0: 1.4349878702340839e-09}, 'suburb': {0.0: 0.9990385566919565, 1.0: 0.000961443308044706}, 'state': {0.0: 0.7998048407886554, 1.0: 0.20019515921134468}, 'address_1': {0.0: 0.9994547031743238, 1.0: 0.000545296825676822}, 'address_2': {0.0: 0.9994547031743238

In [105]:
# evaluate the model
links = cl.predict(features)
print("Predicted number of links:", len(links))

INFO:recordlinkage:Classification - predict matches and non-matches
Predicted number of links: 6253


In [106]:
cm = rl.confusion_matrix(true_links, links, len(candidate_links))
print("Confusion matrix:\n", cm)

Confusion matrix:
 [[ 6236   302]
 [   17 69429]]


In [107]:
# compute the F-score for this classification
fscore = rl.fscore(cm)
print('fscore', fscore)
recall = rl.recall(true_links, links)
print('recall', recall)
precision = rl.precision(true_links, links)
print('precision', precision)

fscore 0.9750605894769759
recall 0.9538085041297033
precision 0.9972813049736127


In [108]:
# Predict the match probability for each pair in the dataset.
probs = cl.prob(features)

INFO:recordlinkage:Classification - compute probabilities
